# ELYZA-tasks-100 による性能評価

In [ ]:
#!pip install accelerate autoawq datasets jsonlines sentencepiece
!pip install accelerate datasets jsonlines sentencepiece
# https://github.com/oobabooga/text-generation-webui/issues/4517#issuecomment-1815279467
!pip install https://github.com/casper-hansen/AutoAWQ/releases/download/v0.1.7/autoawq-0.1.7+cu118-cp310-cp310-linux_x86_64.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.0/154.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.0/229.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

## ELYZA-tasks-100 データセットのダウンロード

モデルのダウンロードには 🤗datasets ライブラリを用いる。

データは `/content/dataset.jsonl` に保存する。

In [ ]:
import json
from pathlib import Path

from datasets import load_dataset

# Load the dataset
ds = load_dataset("elyza/ELYZA-tasks-100")

# Function to convert dataset to JSONL format and print
def dataset_to_jsonl(dataset, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        for entry in dataset:
            # Construct JSON object
            json_obj = {
                "input_text": entry['input'],
                "output_text": entry['output'],
                "eval_aspect": entry['eval_aspect']
            }

            # Write JSON object to file in JSONL format
            json_str = json.dumps(json_obj, ensure_ascii=False)
            file.write(json_str + '\n')

# Convert and write the dataset to a file in JSONL format
path_jsonl = Path("/content/dataset.jsonl")

if (path_jsonl.parent is not None) and (not path_jsonl.parent.exists()):
    path_jsonl.parent.mkdir(parents=True, exist_ok=True)

dataset_to_jsonl(ds["test"], path_jsonl)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

## 評価

ELYZA-tasks-100 データセットを用いて、指定したモデルの評価を実行する。

### プロンプトについて

モデルに回答を生成させるときのプロンプトは、各モデルのモデルカード (🤗HuggingFace の README ページ) に記載されているプロンプトを使用する。

### ハイパーパラメータについて

ハイパーパラメータは 🤗HuggingFace の Code Snippet のとおりとする（公開者が指定したハイパーパラメータを、チューン済みとみなす）。
ただし、長文の回答を要求する設問もあるため、`max_new_tokens` は `1024` に統一する。

In [ ]:
import json
import jsonlines
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Setup model and tokenizer
model_name = "openchat/openchat-3.5-1210"
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    #torch_dtype=torch.float16,
    device_map="auto")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto",
    # trust_remote_code=True, # ⚠️ Microsoft/Phi-1.5 のみ必要
)

def format_prompt(input_text):
    # ✋プロンプトは、モデルに合わせて調整（HuggingFace のモデルカード参照）
    #prompt_template = "GPT4 Correct User: あなたは役立つアシスタントです。ユーザーの質問に回答し、指示に従ってください。<|end_of_turn|>GPT4 Correct Assistant: 承知しました。<|end_of_turn|>GPT4 Correct User: {input}<|end_of_turn|>GPT4 Correct Assistant:"
    prompt_template = "GPT4 Correct User: {input}<|end_of_turn|>GPT4 Correct Assistant:"
    return prompt_template.format(input=input_text)

def generate_text(input_text):
    formatted_prompt = format_prompt(input_text)
    input_ids = tokenizer.encode(
        formatted_prompt,
        add_special_tokens=False,
        return_tensors="pt"
    )

    # Set seed for reproducibility
    seed = 42
    torch.manual_seed(seed)

    # ✋Hparamsは、モデルに合わせて調整（HuggingFace のモデルカード参照）
    tokens = model.generate(
        input_ids.to(device=model.device),
        max_new_tokens=1024,
        do_sample=True,
        #top_p=0.95,
        #temperature=0.7,
        #pad_token_id=tokenizer.pad_token_id,
        #bos_token_id=tokenizer.bos_token_id,
        #eos_token_id=tokenizer.eos_token_id
    )

    # Remove the input tokens from the generated tokens before decoding
    output_tokens = tokens[0][len(input_ids[0]):]
    return tokenizer.decode(output_tokens, skip_special_tokens=True)

def process_dataset(input_file, output_file):
    # Read dataset and generate predictions
    with jsonlines.open(input_file) as reader, jsonlines.open(output_file, mode='w') as writer:
        for obj in reader:
            prompt = obj['input_text']
            generated_text = generate_text(prompt)
            print(generated_text)
            writer.write({"pred": generated_text})

# Process the dataset
input_dataset = '/content/dataset.jsonl'
output_predictions = '/content/preds.jsonl'
process_dataset(input_dataset, output_predictions)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

1. 花粉分布の状況をチェックして外出する：花粉症の方は、天気予報や地方自治体の情報をチェックし、花粉値の高い日は外出するべきです。アプリも役立ちますので、必ずインストールして使いこなそう。

2. 室内活動を大きく含める：花粉症の方は、春の間は室内での活動を大きく心がけることが便riousです。例えば、家族と一緒に家の中でのハビタションや、ハンドメードのアートワークスをしましょう。

3. 適切な薬を用いる: 医師に相談して、適切な薬を使用することで、花粉が生じないようにしましょう。また、薬の使用量や日程を正確に遵守しましょう。


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


1. 立派　A. すばらしい
2. 開拓　D. 新しい領域を切り開く
3. 知識　B. ある事柄について知っている内容。
4. 賞賛　E. 高い評価を与える
5. 味付け　C. 物事に味わいや価値を加える


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


その謎めいた人物は、童願のような白髪の毛毛虫のようなものだった。主人公は驚かせられ、彼がどんな人物であるかを疑問にも迷わせました。

「なぜ私にこの手紙を届けてくれたのかと、どんなことを待っているのか」と主人公はもたらした質問をしました。

謎めいた人物は、首を傾けずに主人公に見返しました。そのつらげる目しはげに、主人公はもちろんその人物に恐れました。

片笑いして言った謎めいた人物は、「私が伝えたい物語があって、あなたがそれを織りなすことです。」と語りました。

その言葉に主人公は驚きました。どんな物語がそれでしょうか。謎めいた人物は眉をかさぐって語った。

その物語は、森の奥深くの洞窟で起きた不思議な出来事でした。長久に過ぎて、その洞窟を持っていた地球人が宇宙に登場し、地球全体が揺れ動き始まりました。

謎めいた人物はその物語を主人公に聞かせ、彼がそれを人々に伝える仕事にこうかった。私たちもその物語を聞きなさい。そして、その物語を織りなし、他の人たちに伝えることでしょうか。

主人公は危機を乗り越し、謎めいた人物の物語を人々に伝え始めました。そして、その物語は世界中に誓い合った生きものたちの手を繋いで、人々が困難を乗り越し、未来へと向かったことに繋がりました。

彼らは、どんな困難にも立ち会い、一緒にこの物語を伝えることで、人々に希べる力を与えました。そしてその物語は永遠に伝わり続けることでしょう。

この物語は、謎めいた人物によって主人公が繋がる人々へと伝えられた物語でした。そしてその物語は時と空间なく、生きと死の邦交で語られる物語でもあった。
1. 昔、あるところに老人と孫がいました。
2. 老人は、広い畑で色々な果物を育てていました。
3. ある日、孫が老人に『こんなに広い畑で、全ての果物を一人で育てるのは大変そうですね』と尋ねました。
4. 老人は微笑みながら次のように答えました。
5. 果物だけではなく、人生も同じです。協力しなくては、全てをこなすことはできません。
